In [1]:
import sqlite3
conn = sqlite3.connect("test.db")
cursor = conn.cursor()

cursor.execute(''' CREATE TABLE Data(
    A INTEGER,
    B INTEGER
);''')

conn.commit()

In [2]:
cursor.execute(''' INSERT INTO Data(A, B) VALUES
            (1, 3),
            (3, 7),
            (5, 9),
            (7, 11),
            (9, 15);''')
conn.commit()

Câu 1

In [6]:
cursor.execute('''WITH DataCTE AS (
        SELECT * FROM Data
    ),
    Sums AS (
        SELECT 
            SUM(A) AS sum_a,
            SUM(B) AS sum_b,
            SUM(A * B) AS sum_ab,
            SUM(POWER(A, 2)) AS sum_a_sq,
            SUM(POWER(B, 2)) AS sum_b_sq,
            COUNT(*) AS n
        FROM DataCTE
    )
SELECT 
    (n * sum_ab - sum_a * sum_b) / 
    (SQRT(n * sum_a_sq - POWER(sum_a, 2)) * SQRT(n * sum_b_sq - POWER(sum_b, 2))) AS r_AB
FROM Sums;''')
print(cursor.fetchall())
conn.commit()

[(0.9899494936611665,)]


Câu 2


In [34]:
cursor.execute('''CREATE TABLE car_data(
    day TEXT,
    model TEXT,
    score FLOAT
);''')
conn.commit()

In [35]:
cursor.execute('''INSERT INTO car_data VALUES
    ('Day1', 'A', 8.0),
    ('Day1', 'B', 9.0),
    ('Day1', 'C', 7.0),
    ('Day2', 'A', 7.5),
    ('Day2', 'B', 8.5),
    ('Day2', 'C', 7.0),
    ('Day3', 'A', 6.0),
    ('Day3', 'B', 7.0),
    ('Day3', 'C', 8.0),
    ('Day4', 'A', 7.0),
    ('Day4', 'B', 6.0),
    ('Day4', 'C', 5.0);''')
conn.commit()

In [36]:
cursor.execute('''
-- Thực hiện kiểm định Chi-square
WITH Observed AS (
    SELECT 
        model,
        day,
        score as freq
    FROM car_data
),
-- Tính tổng hàng (tổng cho mỗi ngày)
RowTotals AS (
    SELECT 
        day,
        SUM(freq) AS total_day
    FROM Observed
    GROUP BY day
),
-- Tính tổng cột (tổng cho từng mô hình)
ColumnTotals AS (
    SELECT 
        model,
        SUM(freq) AS total_model
    FROM Observed
    GROUP BY model
),
-- Tính tần suất dự kiến
Expected AS (
    SELECT 
        o.model,
        o.day,
        (ct.total_model * rt.total_day) / (SELECT SUM(freq) FROM Observed) AS expected_freq
    FROM Observed o
    JOIN RowTotals rt ON o.day = rt.day
    JOIN ColumnTotals ct ON o.model = ct.model
)
-- Tính thống kê chi bình phương
SELECT 
    ROUND(SUM(POWER(o.freq - e.expected_freq, 2) / e.expected_freq), 4) AS chi_square
FROM Observed o
JOIN Expected e ON o.model = e.model AND o.day = e.day;
''')
print(cursor.fetchall())
conn.commit()


[(0.8264,)]


In [ ]:
# Bậc tự do
cursor.execute('''SELECT 
    (COUNT(DISTINCT day) - 1) * (COUNT(DISTINCT model) - 1) AS degrees_of_freedom
FROM car_data;''')
print(cursor.fetchall())


[(6,)]


0.8264 < 12.592 (giá trị tới hạn)=> không thể bác bỏ giả thuyết H0

Vì vậy: 

không có sự khác biệt đáng kể giữa các mẫu xe A, B và C(Hiệu suất của 3 mẫu xe khá tương đồng nhau)

Điểm số không phụ thuộc đáng kể vào ngày thử nghiệm (Thời gian thử nghiệm không ảnh hưởng đáng kể đến kết quả)

Sự khác biệt trong điểm số có thể do ngẫu nhiên



Câu 3

In [13]:
cursor.execute("""
CREATE TABLE flights (
    id INTEGER PRIMARY KEY,
    departure_time INTEGER
);""")

data = [(1, 830), (2, 1445), (3, 8), (4, 6), (5, 15), (6, 10), (7, 11), (8, 12), (9, 13), (10, 14)]
cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", data)
conn.commit()

In [16]:
cursor.execute('''SELECT departure_time,
    TIME(
        substr('0000' || departure_time, -4, 2) || ':' ||
        substr('0000' || departure_time, -2) || ':00'
    ) AS formatted_time
FROM flights;''')
print(cursor.fetchall())
conn.commit()

[(830, '08:30:00'), (1445, '14:45:00'), (8, '00:08:00'), (6, '00:06:00'), (15, '00:15:00'), (10, '00:10:00'), (11, '00:11:00'), (12, '00:12:00'), (13, '00:13:00'), (14, '00:14:00')]


Câu 4

In [17]:
cursor.execute('''
CREATE TABLE test_data (
    id INTEGER PRIMARY KEY,
    value FLOAT
);''')

cursor.execute('''
INSERT INTO test_data (value) 
VALUES 
    (10), (12), 
    (13), (14), 
    (15), (16), 
    (17), (18), 
    (100), (11);''')

conn.commit()

In [18]:
cursor.execute('''WITH stats AS (
    SELECT AVG(value) as median_val
    FROM (
        SELECT value
        FROM test_data
        ORDER BY value
        LIMIT 2 - (SELECT COUNT(*) FROM test_data) % 2
        OFFSET (SELECT (COUNT(*) - 1) / 2 FROM test_data)
    )
),

mad_calc AS (
    SELECT AVG(ABS(value - median_val)) as mad
    FROM test_data, stats
)

SELECT value, 
    ABS(value - (SELECT median_val FROM stats)) as deviation,
    (SELECT mad FROM mad_calc) as mad
FROM test_data
WHERE ABS(value - (SELECT median_val FROM stats)) > 
      1.5 * (SELECT mad FROM mad_calc)
ORDER BY value;''')

print(cursor.fetchall())
conn.commit()

[(100.0, 85.5, 10.6)]


Câu 5

In [ ]:
cursor.execute(''' CREATE TABLE Patient (
    patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
    last_name TEXT,
    height FLOAT,
    weight FLOAT
);''')

cursor.execute('''INSERT INTO Patient (last_name, height, weight) VALUES 
('Nguyen', 170, 60),
('Tran', 165, 50),
('Le', 182, 70),
('Pham', 157, 40),
('Hoang', 177, 80),
('Smith', 170, 65),
('Johnson', 180, 75),
('Williams', 175, 70),
('Jones', 160, 55),
('Brown', 172, 68),
('Davis', 169, 62),
('Garcia', 178, 72),
('Rodriguez', 174, 66),
('Martinez', 163, 58),
('Hernandez', 167, 64),
('Lopez', 171, 63),
('Smith', 168, 65)''')
conn.commit()

In [30]:
cursor.execute('''SELECT p1.last_name, p1.weight, p2.last_name, p2.weight
FROM Patient p1, Patient p2
WHERE p1.last_name = p2.last_name 
AND p1.weight = p2.weight
AND p1.patient_id < p2.patient_id;''')
print(cursor.fetchall())
conn.commit()

[('Smith', 65.0, 'Smith', 65.0)]
